<a href="https://colab.research.google.com/github/nabazar/AI-Aimbot/blob/main/DQN4XRLv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!git clone  https://github.com/WeihaoTan/gym-macro-overcooked.git

fatal: destination path 'gym-macro-overcooked' already exists and is not an empty directory.


In [22]:
#
try:
    import shap
except ImportError:
  !pip install shap
  import  shap

In [23]:

#
try:
    import lime
except ImportError:
  !pip install lime
  import  lime

In [24]:
import gym
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import PartialDependenceDisplay

import pygame
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# from .render.game import Game
from gym import spaces
# from .items import Tomato, Lettuce, Onion, Plate, Knife, Delivery, Agent, Food
import copy

#Import Libraries

In [25]:
import os
from typing import Dict, List, Tuple
import pickle
import numpy.matlib
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import clear_output
from google.colab import files

In [26]:
import random
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
from IPython.display import clear_output
import matplotlib.pyplot as plt
import argparse
import os
from IPython.display import clear_output
import matplotlib.pyplot as plt
import time
from sklearn import preprocessing
from gym import spaces
from gym.spaces import Tuple , Box


import random
from collections import deque
from typing import Deque, Dict, List, Tuple

import gym
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from IPython.display import clear_output
from torch.distributions import Normal

In [ ]:
# @title PPO agent
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

class PPOAgent:
    def __init__(self, env, state_size, action_size, lr=3e-4, gamma=0.99, clip_ratio=0.2, update_epochs=10, batch_size=64):
        self.env = env
        self.state_size = state_size
        self.action_size = action_size
        self.lr = lr
        self.gamma = gamma
        self.clip_ratio = clip_ratio
        self.update_epochs = update_epochs
        self.batch_size = batch_size

        self.policy = PolicyNet(self.state_size, self.action_size)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=self.lr)

    def select_action(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0)
        probs = self.policy(state).squeeze(0)
        dist = Categorical(probs)
        action = dist.sample().item()
        return action

    def update(self, states, actions, rewards, dones, next_states):
        """Update the policy network"""
        states = torch.from_numpy(np.array(states)).float()
        actions = torch.from_numpy(np.array(actions)).long()
        rewards = torch.from_numpy(np.array(rewards)).float()
        dones = torch.from_numpy(np.array(dones)).float()
        next_states = torch.from_numpy(np.array(next_states)).float()

        # Calculate discounted rewards
        returns = self.calculate_returns(rewards, dones)

        for _ in range(self.update_epochs):
            # Get the current action probabilities
            probs = self.policy(states).gather(1, actions.unsqueeze(1)).squeeze(1)

            # Get the old action probabilities
            with torch.no_grad():
                old_probs = self.policy(states).gather(1, actions.unsqueeze(1)).squeeze(1)

            # Calculate the ratio between new and old probabilities
            ratios = probs / old_probs

            # Calculate the surrogate loss
            advantage = returns - self.policy.value(states).squeeze()
            surr1 = ratios * advantage
            surr2 = torch.clamp(ratios, 1 - self.clip_ratio, 1 + self.clip_ratio) * advantage
            loss = -torch.mean(torch.min(surr1, surr2))

            # Update the policy network
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

    def calculate_returns(self, rewards, dones):
        """Calculate the discounted returns"""
        returns = []
        R = 0
        for r, d in zip(reversed(rewards), reversed(dones)):
            R = r + self.gamma * R * (1 - d)
            returns.insert(0, R)
        return torch.tensor(returns)

class PolicyNet(nn.Module):
    def __init__(self, state_size, action_size):
        super().__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc_pi = nn.Linear(64, action_size)
        self.fc_v = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        pi = torch.softmax(self.fc_pi(x), dim=-1)
        v = self.fc_v(x)
        return pi, v

    def value(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc_v(x)

#Policy Gradient

In [ ]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

class PolicyGradientAgent:
    def __init__(self, env, state_size, action_size, lr=1e-3, gamma=0.99):
        self.env = env
        self.state_size = state_size
        self.action_size = action_size
        self.lr = lr
        self.gamma = gamma

        self.policy = PolicyNet(self.state_size, self.action_size)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=self.lr)

    def select_action(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0)
        probs, _ = self.policy(state)
        dist = Categorical(probs)
        action = dist.sample().item()
        return action

    def update(self, states, actions, rewards):
        """Update the policy network"""
        states = torch.from_numpy(np.array(states)).float()
        actions = torch.from_numpy(np.array(actions)).long()
        rewards = torch.from_numpy(np.array(rewards)).float()

        # Calculate the discounted rewards
        returns = self.calculate_returns(rewards)

        # Compute the policy loss
        probs, _ = self.policy(states)
        dist = Categorical(probs)
        log_probs = dist.log_prob(actions)
        policy_loss = -torch.mean(log_probs * returns)

        # Update the policy network
        self.optimizer.zero_grad()
        policy_loss.backward()
        self.optimizer.step()

    def calculate_returns(self, rewards):
        """Calculate the discounted returns"""
        returns = []
        R = 0
        for r in reversed(rewards):
            R = r + self.gamma * R
            returns.insert(0, R)
        return torch.tensor(returns)

class PolicyNet(nn.Module):
    def __init__(self, state_size, action_size):
        super().__init__()
        self.fc1 = nn.Linear(state_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc_pi = nn.Linear(64, action_size)
        self.fc_v = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        pi = torch.softmax(self.fc_pi(x), dim=-1)
        v = self.fc_v(x)
        return pi, v

    def value(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc_v(x)

In [28]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

class PolicyGradientAgent:
    def __init__(self, env, lr=0.01, gamma=0.99):
        self.env = env
        self.state_size = env.obs_space.shape[0]
        self.action_size = env.action_space.n

        self.policy_net = nn.Sequential(
            nn.Linear(self.state_size, 64),
            nn.ReLU(),
            nn.Linear(64, self.action_size),
            nn.Softmax(dim=1)
        )

        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=lr)
        self.gamma = gamma

    def select_action(self, state):
        state = torch.from_numpy(state).float().unsqueeze(0)
        probs = self.policy_net(state)
        dist = Categorical(probs)
        action = dist.sample()
        return action.item()

    def update_policy(self, rewards, states, actions):
        discounted_rewards = self.discount_rewards(rewards)
        states = torch.from_numpy(np.array(states)).float()
        actions = torch.tensor(actions)

        probs = self.policy_net(states)
        dist = Categorical(probs)
        log_probs = dist.log_prob(actions)

        loss = -torch.mean(log_probs * discounted_rewards)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()



    def discount_rewards(self, rewards):
        discounted_rewards = np.zeros_like(rewards)
        running_add = 0
        for i in reversed(range(len(rewards))):
            running_add = running_add * self.gamma + rewards[i]
            discounted_rewards[i] = running_add
        return (discounted_rewards - np.mean(discounted_rewards)) / (np.std(discounted_rewards) + 1e-8)


In [29]:
# @title Replay Buffer
class ReplayBuffer:
    """A simple numpy replay buffer."""

    def __init__(self, obs_dim: int, size: int, batch_size: int = 32):
        self.obs_buf = np.zeros([size, obs_dim], dtype=np.float32)
        self.next_obs_buf = np.zeros([size, obs_dim], dtype=np.float32)
        self.acts_buf = np.zeros([size], dtype=np.float32)
        self.rews_buf = np.zeros([size], dtype=np.float32)
        self.done_buf = np.zeros(size, dtype=np.float32)
        self.max_size, self.batch_size = size, batch_size
        self.ptr, self.size, = 0, 0

    def store(
        self,
        obs: np.ndarray,
        act: np.ndarray,
        rew: float,
        next_obs: np.ndarray,
        done: bool,
    ):

        self.obs_buf[self.ptr] = obs
        self.next_obs_buf[self.ptr] = next_obs
        self.acts_buf[self.ptr] = act
        self.rews_buf[self.ptr] = rew
        self.done_buf[self.ptr] = done
        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

    def sample_batch(self) -> Dict[str, np.ndarray]:
        idxs = np.random.choice(self.size, size=self.batch_size, replace=False)
        return dict(obs=self.obs_buf[idxs],
                    next_obs=self.next_obs_buf[idxs],
                    acts=self.acts_buf[idxs],
                    rews=self.rews_buf[idxs],
                    done=self.done_buf[idxs])

    def __len__(self) -> int:
        return self.size

In [30]:
# @title Network
class Network(nn.Module):
    def __init__(self, in_dim: int, out_dim: int):
        """Initialization."""
        super(Network, self).__init__()
        print('out_dim',out_dim)

        self.layers = nn.Sequential(
            nn.Linear(in_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, out_dim)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward method implementation."""
        return self.layers(x)

In [31]:
# @title DQN agent
class DQNAgent:
    """DQN Agent interacting with environment.

    Attribute:
        env (gym.Env): openAI Gym environment
        memory (ReplayBuffer): replay memory to store transitions
        batch_size (int): batch size for sampling
        epsilon (float): parameter for epsilon greedy policy
        epsilon_decay (float): step size to decrease epsilon
        max_epsilon (float): max value of epsilon
        min_epsilon (float): min value of epsilon
        target_update (int): period for target model's hard update
        gamma (float): discount factor
        dqn (Network): model to train and select actions
        dqn_target (Network): target model to update
        optimizer (torch.optim): optimizer for training dqn
        transition (list): transition information including
                           state, action, reward, next_state, done
    """

    def __init__(
        self,
        env,
        memory_size: int,
        batch_size: int,
        target_update: int,
        epsilon_decay: float,
        max_epsilon: float = 1.0,
        min_epsilon: float = 0.1,
        gamma: float = 0.99,
    ):
        """Initialization.

        Args:
            env (gym.Env): openAI Gym environment
            memory_size (int): length of memory
            batch_size (int): batch size for sampling
            target_update (int): period for target model's hard update
            epsilon_decay (float): step size to decrease epsilon
            lr (float): learning rate
            max_epsilon (float): max value of epsilon
            min_epsilon (float): min value of epsilon
            gamma (float): discount factor
        """
        obs_dim = env.obs_space.shape[0]
        action_dim = env.action_space.n

        self.env = env
        self.memory = ReplayBuffer(obs_dim, memory_size, batch_size)
        self.batch_size = batch_size
        self.epsilon = max_epsilon
        self.epsilon_decay = epsilon_decay

        self.max_epsilon = max_epsilon
        self.min_epsilon = min_epsilon
        self.target_update = target_update
        self.gamma = gamma

        # device: cpu / gpu
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu"
        )
        print(self.device)

        # networks: dqn, dqn_target
        self.dqn = Network(obs_dim, action_dim).to(self.device)
        self.dqn_target = Network(obs_dim, action_dim).to(self.device)
        self.dqn_target.load_state_dict(self.dqn.state_dict())
        self.dqn_target.eval()

        # optimizer
        self.optimizer = optim.Adam(self.dqn.parameters())

        # transition to store in memory
        self.transition = list()

        # mode: train / test
        self.is_test = False

    def reset(self):
      state=self.env.reset()
      state=np.concatenate(state,axis=None)
      return state

    def select_action(self, state: np.ndarray) -> np.ndarray:
        """Select an action from the input state."""
        # epsilon greedy policy
        if  self.is_test:
          selected_action = self.dqn(torch.FloatTensor(state).to(self.device)).argmax()
          selected_action = selected_action.detach().cpu().numpy()
        else:
          # self.epsilon=0.3
          if self.epsilon > np.random.random():
              selected_action = self.env.action_space.sample()
          else:
              selected_action = self.dqn(
                  torch.FloatTensor(state).to(self.device)
              ).argmax()
              selected_action = selected_action.detach().cpu().numpy()

        if not self.is_test:
            self.transition = [state, selected_action]

        return selected_action

    def step(self, action):
        """Take an action and return the response of the env."""
        next_state, reward,done,info = self.env.step(action)
        next_state=np.concatenate(next_state,axis=None)
        reward=np.sum(reward)


        if not self.is_test:
            self.transition += [reward, next_state, done]
            self.memory.store(*self.transition)

        return next_state, reward, done,info

    def update_model(self) -> torch.Tensor:
        """Update the model by gradient descent."""
        samples = self.memory.sample_batch()

        loss = self._compute_dqn_loss(samples)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss.item()

    def _compute_dqn_loss(self, samples: Dict[str, np.ndarray]) -> torch.Tensor:
        """Return dqn loss."""
        device = self.device  # for shortening the following lines
        state = torch.FloatTensor(samples["obs"]).to(device)
        next_state = torch.FloatTensor(samples["next_obs"]).to(device)
        action = torch.LongTensor(samples["acts"].reshape(-1, 1)).to(device)
        reward = torch.FloatTensor(samples["rews"].reshape(-1, 1)).to(device)
        done = torch.FloatTensor(samples["done"].reshape(-1, 1)).to(device)

        # G_t   = r + gamma * v(s_{t+1})  if state != Terminal
        #       = r                       otherwise
        curr_q_value = self.dqn(state).gather(1, action)
        next_q_value = self.dqn_target(
            next_state
        ).max(dim=1, keepdim=True)[0].detach()
        mask = 1 - done
        target = (reward + self.gamma * next_q_value * mask).to(self.device)

        # calculate dqn loss
        loss = F.smooth_l1_loss(curr_q_value, target)

        return loss

    def _target_hard_update(self):
        """Hard update: target <- local."""
        self.dqn_target.load_state_dict(self.dqn.state_dict())


In [32]:
class Color:
    BLACK = (0, 0, 0)
    FLOOR = (245, 230, 210)  # light gray
    COUNTER = (220, 170, 110)   # tan/gray
    COUNTER_BORDER = (114, 93, 51)  # darker tan
    DELIVERY = (96, 96, 96)  # grey

KeyToTuple = {
    pygame.K_UP    : ( 0, -1),  #273
    pygame.K_DOWN  : ( 0,  1),  #274
    pygame.K_RIGHT : ( 1,  0),  #275
    pygame.K_LEFT  : (-1,  0),  #276
}

In [33]:
# @title Defining Items
#!/usr/bin/python

import numpy as np

class Item(object):
    def __init__(self, pos_x, pos_y):
        self.x = pos_x
        self.y = pos_y

class MovableItem(Item):
    def __init__(self, pos_x, pos_y):
        super().__init__(pos_x, pos_y)
        self.initial_x = pos_x
        self.initial_y = pos_y

    def move(self, x, y):
        self.x = x
        self.y = y

    def refresh(self):
        self.x = self.initial_x
        self.y = self.initial_y

class Food(MovableItem):
    # 0 for unchoopped 1 for chopped
    def __init__(self, pos_x, pos_y, chopped = False):
        super().__init__(pos_x, pos_y)
        self.chopped = chopped
        self.cur_chopped_times = 0
        self.required_chopped_times = 3

    def chop(self):
        if not self.chopped:
            self.cur_chopped_times += 1
            if self.cur_chopped_times >= self.required_chopped_times:
                self.chopped = True

    def refresh(self):
        self.x = self.initial_x
        self.y = self.initial_y
        self.chopped = False
        self.cur_chopped_times = 0

class Tomato(Food):
    def __init__(self, pos_x, pos_y):
        super().__init__(pos_x, pos_y)
        self.rawName = "tomato"

    @property
    def name(self):
        if self.chopped:
            return "ChoppedTomato"
        else:
            return "FreshTomato"

class Lettuce(Food):
    def __init__(self, pos_x, pos_y):
        super().__init__(pos_x, pos_y)
        self.rawName = "lettuce"

    @property
    def name(self):
        if self.chopped:
            return "ChoppedLettuce"
        else:
            return "FreshLettuce"

class Onion(Food):
    def __init__(self, pos_x, pos_y):
        super().__init__(pos_x, pos_y)
        self.rawName = "onion"

    @property
    def name(self):
        if self.chopped:
            return "ChoppedOnion"
        else:
            return "FreshOnion"

class FixedItem(Item):
    def __init__(self, pos_x, pos_y, holding = None):
        super().__init__(pos_x, pos_y)
        self.holding = holding

    def hold(self, items):
        self.holding = items

    def release(self):
        self.holding = None

class Knife(FixedItem):
    def __init__(self, pos_x, pos_y, holding = None):
        super().__init__(pos_x, pos_y, holding)
        self.rawName = "knife"

    @property
    def name(self):
        return "cutboard"

class Delivery(FixedItem):
    def __init__(self, pos_x, pos_y, holding = None):
        super().__init__(pos_x, pos_y, holding)
        self.rawName = "delivery"

    @property
    def name(self):
        return "delivery"


class Plate(MovableItem):
    def __init__(self, pos_x, pos_y, containing = None):
        super().__init__(pos_x, pos_y)
        self.containing = containing
        self.rawName = "plate"

    def contain(self, items):
        if self.containing:
            self.containing.append(items)
        else:
            self.containing = [items]
        for item in self.containing:
            item.move(self.x, self.y)

    def move(self, x, y):
        super().move(x, y)
        if self.containing:
            for item in self.containing:
                item.move(x, y)

    def release(self):
        self.containing = None

    @property
    def name(self):
        return "plate"

    @property
    def containedName(self):
        dishName = ""
        foodList = [Lettuce, Onion, Tomato]
        foodInPlate = [-1] * len(foodList)

        for f in range(len(self.containing)):
            for i in range(len(foodList)):
                if isinstance(self.containing[f], foodList[i]):
                    foodInPlate[i] = f
        for i in range(len(foodList)):
            if foodInPlate[i] > -1:
                dishName += self.containing[foodInPlate[i]].name + "-"
        return dishName[:-1]


class Agent(MovableItem):
    def __init__(self, pos_x, pos_y, holding = None, color = None):
        super().__init__(pos_x, pos_y)
        self.holding = holding
        self.color = color
        self.moved = False
        self.obs = None
        self.pomap = None
        self.rawName = "agent"

    def pickup(self,item):
        self.holding = item
        item.move(self.x, self.y)

    def putdown(self, x, y):
        self.holding.move(x, y)
        self.holding = None

    def move(self, x, y):
        super().move(x, y)
        self.moved = True
        if self.holding:
            self.holding.move(x, y)

In [34]:
# @title Game
import os
import sys

import pygame
import numpy as np
# from .utils import *
# from ..items import Tomato, Lettuce, Plate, Knife, Delivery, Agent, Food

# graphics_dir = os.path.abspath(os.path.join(os.path.dirname(__file__), 'graphics'))
graphics_dir ="/content/gym-macro-overcooked/gym_macro_overcooked/render/graphics"
_image_library = {}

ITEMNAME = ["space", "counter", "agent", "tomato", "lettuce", "plate", "knife", "delivery", "onion"]
ITEMIDX= {"space": 0, "counter": 1, "agent": 2, "tomato": 3, "lettuce": 4, "plate": 5, "knife": 6, "delivery": 7, "onion": 8}

def get_image(path):
    global _image_library
    image = _image_library.get(path)
    if image == None:
        canonicalized_path = path.replace('/', os.sep).replace('\\', os.sep)
        image = pygame.image.load(canonicalized_path)
        _image_library[path] = image
    return image


class Game:
    def __init__(self, env):
        self._running = True
        self.env = env

        # Visual parameters
        self.scale = 80   # num pixels per tile
        self.holding_scale = 0.5
        self.container_scale = 0.7
        self.width = self.scale * self.env.xlen
        self.height = self.scale * self.env.ylen
        self.tile_size = (self.scale, self.scale)
        self.holding_size = tuple((self.holding_scale * np.asarray(self.tile_size)).astype(int))
        self.container_size = tuple((self.container_scale * np.asarray(self.tile_size)).astype(int))
        self.holding_container_size = tuple((self.container_scale * np.asarray(self.holding_size)).astype(int))

        pygame.init()

    def on_init(self):
        pygame.init()
        if self.play:
            self.screen = pygame.display.set_mode((self.width, self.height))
        else:
            # Create a hidden surface
            self.screen = pygame.Surface((self.width, self.height))
        self._running = True


    def on_event(self, event):
        if event.type == pygame.QUIT:
            self._running = False

    def on_render(self):
        self.screen = pygame.display.set_mode((self.width, self.height))
        self.screen.fill(Color.FLOOR)
        for x in range(self.env.xlen):
            for y in range(self.env.ylen):
                sl = self.scaled_location((y, x))
                if self.env.map[x][y] == ITEMIDX["counter"]:
                    fill = pygame.Rect(sl[0], sl[1], self.scale, self.scale)
                    pygame.draw.rect(self.screen, Color.COUNTER, fill)
                    pygame.draw.rect(self.screen, Color.COUNTER_BORDER, fill, 1)
                elif self.env.map[x][y] == ITEMIDX["delivery"]:
                    fill = pygame.Rect(sl[0], sl[1], self.scale, self.scale)
                    pygame.draw.rect(self.screen, Color.DELIVERY, fill)
                    self.draw('delivery', self.tile_size, sl)
                    for k in self.env.delivery:
                        if k.x == x and k.y == y:
                            if k.holding:
                                self.draw(k.holding.name, self.tile_size, sl)
                                if k.holding.name == "plate":
                                    if k.holding.containing:
                                        self.draw(k.holding.containedName, self.container_size, self.container_location((y, x)))
                elif self.env.map[x][y] == ITEMIDX["knife"]:
                    fill = pygame.Rect(sl[0], sl[1], self.scale, self.scale)
                    pygame.draw.rect(self.screen, Color.COUNTER, fill)
                    pygame.draw.rect(self.screen, Color.COUNTER_BORDER, fill, 1)
                    self.draw('cutboard', self.tile_size, sl)
                    for k in self.env.knife:
                        if k.x == x and k.y == y:
                            if k.holding:
                                self.draw(k.holding.name, self.tile_size, sl)
                                if k.holding.name == "plate":
                                    if k.holding.containing:
                                        self.draw(k.holding.containedName, self.container_size, self.container_location((y, x)))
                elif self.env.map[x][y] == ITEMIDX["tomato"]:
                    fill = pygame.Rect(sl[0], sl[1], self.scale, self.scale)
                    pygame.draw.rect(self.screen, Color.COUNTER, fill)
                    pygame.draw.rect(self.screen, Color.COUNTER_BORDER, fill, 1)
                    for t in self.env.tomato:
                        if t.x == x and t.y == y:
                            self.draw(t.name, self.tile_size, sl)
                elif self.env.map[x][y] == ITEMIDX["lettuce"]:
                    fill = pygame.Rect(sl[0], sl[1], self.scale, self.scale)
                    pygame.draw.rect(self.screen, Color.COUNTER, fill)
                    pygame.draw.rect(self.screen, Color.COUNTER_BORDER, fill, 1)
                    for l in self.env.lettuce:
                        if l.x == x and l.y == y:
                            self.draw(l.name, self.tile_size, sl)
                elif self.env.map[x][y] == ITEMIDX["onion"]:
                    fill = pygame.Rect(sl[0], sl[1], self.scale, self.scale)
                    pygame.draw.rect(self.screen, Color.COUNTER, fill)
                    pygame.draw.rect(self.screen, Color.COUNTER_BORDER, fill, 1)
                    for l in self.env.onion:
                        if l.x == x and l.y == y:
                            self.draw(l.name, self.tile_size, sl)
                elif self.env.map[x][y] == ITEMIDX["plate"]:
                    fill = pygame.Rect(sl[0], sl[1], self.scale, self.scale)
                    pygame.draw.rect(self.screen, Color.COUNTER, fill)
                    pygame.draw.rect(self.screen, Color.COUNTER_BORDER, fill, 1)
                    self.draw('plate', self.tile_size, sl)
                    for p in self.env.plate:
                        if p.x == x and p.y == y:
                            if p.containing:
                                self.draw(p.containedName, self.container_size, self.container_location((y, x)))
                elif self.env.map[x][y] == ITEMIDX["agent"]:
                    for agent in self.env.agent:
                        if agent.x == x and agent.y == y:
                            self.draw('agent-{}'.format(agent.color), self.tile_size, sl)
                            if agent.holding:
                                if isinstance(agent.holding, Plate):
                                    self.draw('plate', self.holding_size, self.holding_location((y, x)))
                                    if agent.holding.containing:
                                        self.draw(agent.holding.containedName, self.holding_container_size, self.holding_container_location((y, x)))
                                else:
                                    self.draw(agent.holding.name, self.holding_size, self.holding_location((y, x)))
        pygame.display.flip()
        pygame.display.update()

        img_int = pygame.PixelArray(self.screen)
        img_rgb = np.zeros([img_int.shape[1], img_int.shape[0], 3], dtype=np.uint8)
        for i in range(img_int.shape[0]):
            for j in range(img_int.shape[1]):
                color = pygame.Color(img_int[i][j])
                img_rgb[j, i, 0] = color[1]
                img_rgb[j, i, 1] = color[2]
                img_rgb[j, i, 2] = color[3]

        del img_int
        return img_rgb

    def draw(self, path, size, location):
        image_path = '{}/{}.png'.format(graphics_dir, path)
        image = pygame.transform.scale(get_image(image_path), size)
        self.screen.blit(image, location)

    def scaled_location(self, loc):
        """Return top-left corner of scaled location given coordinates loc, e.g. (3, 4)"""
        return tuple(self.scale * np.asarray(loc))

    def holding_location(self, loc):
        """Return top-left corner of location where agent holding will be drawn (bottom right corner) given coordinates loc, e.g. (3, 4)"""
        scaled_loc = self.scaled_location(loc)
        return tuple((np.asarray(scaled_loc) + self.scale*(1-self.holding_scale)).astype(int))

    def container_location(self, loc):
        """Return top-left corner of location where contained (i.e. plated) object will be drawn, given coordinates loc, e.g. (3, 4)"""
        scaled_loc = self.scaled_location(loc)
        return tuple((np.asarray(scaled_loc) + self.scale*(1-self.container_scale)/2).astype(int))

    def holding_container_location(self, loc):
        """Return top-left corner of location where contained, held object will be drawn given coordinates loc, e.g. (3, 4)"""
        scaled_loc = self.scaled_location(loc)
        factor = (1-self.holding_scale) + (1-self.container_scale)/2*self.holding_scale
        return tuple((np.asarray(scaled_loc) + self.scale*factor).astype(int))

    def on_cleanup(self):
        pygame.display.quit()
        pygame.quit()

    def get_image_obs(self):
        self.screen = pygame.Surface((self.width, self.height))
        self.screen.fill(Color.FLOOR)
        for x in range(self.env.xlen):
            for y in range(self.env.ylen):
                sl = self.scaled_location((y, x))
                if self.env.map[x][y] == ITEMIDX["counter"]:
                    fill = pygame.Rect(sl[0], sl[1], self.scale, self.scale)
                    pygame.draw.rect(self.screen, Color.COUNTER, fill)
                    pygame.draw.rect(self.screen, Color.COUNTER_BORDER, fill, 1)
                elif self.env.map[x][y] == ITEMIDX["delivery"]:
                    fill = pygame.Rect(sl[0], sl[1], self.scale, self.scale)
                    pygame.draw.rect(self.screen, Color.DELIVERY, fill)
                    self.draw('delivery', self.tile_size, sl)
                    for k in self.env.delivery:
                        if k.x == x and k.y == y:
                            if k.holding:
                                self.draw(k.holding.name, self.tile_size, sl)
                                if k.holding.name == "plate":
                                    if k.holding.containing:
                                        self.draw(k.holding.containedName, self.container_size, self.container_location((y, x)))
                elif self.env.map[x][y] == ITEMIDX["knife"]:
                    fill = pygame.Rect(sl[0], sl[1], self.scale, self.scale)
                    pygame.draw.rect(self.screen, Color.COUNTER, fill)
                    pygame.draw.rect(self.screen, Color.COUNTER_BORDER, fill, 1)
                    self.draw('cutboard', self.tile_size, sl)
                    for k in self.env.knife:
                        if k.x == x and k.y == y:
                            if k.holding:
                                self.draw(k.holding.name, self.tile_size, sl)
                                if k.holding.name == "plate":
                                    if k.holding.containing:
                                        self.draw(k.holding.containedName, self.container_size, self.container_location((y, x)))
                elif self.env.map[x][y] == ITEMIDX["tomato"]:
                    fill = pygame.Rect(sl[0], sl[1], self.scale, self.scale)
                    pygame.draw.rect(self.screen, Color.COUNTER, fill)
                    pygame.draw.rect(self.screen, Color.COUNTER_BORDER, fill, 1)
                    for t in self.env.tomato:
                        if t.x == x and t.y == y:
                            self.draw(t.name, self.tile_size, sl)
                elif self.env.map[x][y] == ITEMIDX["lettuce"]:
                    fill = pygame.Rect(sl[0], sl[1], self.scale, self.scale)
                    pygame.draw.rect(self.screen, Color.COUNTER, fill)
                    pygame.draw.rect(self.screen, Color.COUNTER_BORDER, fill, 1)
                    for l in self.env.lettuce:
                        if l.x == x and l.y == y:
                            self.draw(l.name, self.tile_size, sl)
                elif self.env.map[x][y] == ITEMIDX["onion"]:
                    fill = pygame.Rect(sl[0], sl[1], self.scale, self.scale)
                    pygame.draw.rect(self.screen, Color.COUNTER, fill)
                    pygame.draw.rect(self.screen, Color.COUNTER_BORDER, fill, 1)
                    for l in self.env.onion:
                        if l.x == x and l.y == y:
                            self.draw(l.name, self.tile_size, sl)
                elif self.env.map[x][y] == ITEMIDX["plate"]:
                    fill = pygame.Rect(sl[0], sl[1], self.scale, self.scale)
                    pygame.draw.rect(self.screen, Color.COUNTER, fill)
                    pygame.draw.rect(self.screen, Color.COUNTER_BORDER, fill, 1)
                    self.draw('plate', self.tile_size, sl)
                    for p in self.env.plate:
                        if p.x == x and p.y == y:
                            if p.containing:
                                self.draw(p.containedName, self.container_size, self.container_location((y, x)))
                elif self.env.map[x][y] == ITEMIDX["agent"]:
                    for agent in self.env.agent:
                        if agent.x == x and agent.y == y:
                            self.draw('agent-{}'.format(agent.color), self.tile_size, sl)
                            if agent.holding:
                                if isinstance(agent.holding, Plate):
                                    self.draw('plate', self.holding_size, self.holding_location((y, x)))
                                    if agent.holding.containing:
                                        self.draw(agent.holding.containedName, self.holding_container_size, self.holding_container_location((y, x)))
                                else:
                                    self.draw(agent.holding.name, self.holding_size, self.holding_location((y, x)))

        img_int = pygame.PixelArray(self.screen)
        img_rgb = np.zeros([img_int.shape[1], img_int.shape[0], 3], dtype=np.uint8)
        for i in range(img_int.shape[0]):
            for j in range(img_int.shape[1]):
                color = pygame.Color(img_int[i][j])
                img_rgb[j, i, 0] = color[1]
                img_rgb[j, i, 1] = color[2]
                img_rgb[j, i, 2] = color[3]
        # del img_int
        return img_rgb

In [35]:
# @title Environment
DIRECTION = [(0,1), (1,0), (0,-1), (-1,0)]
ITEMNAME = ["space", "counter", "agent", "tomato", "lettuce", "plate", "knife", "delivery", "onion"]
ITEMIDX= {"space": 0, "counter": 1, "agent": 2, "tomato": 3, "lettuce": 4, "plate": 5, "knife": 6, "delivery": 7, "onion": 8}
AGENTCOLOR = ["blue", "magenta", "green", "yellow"]
TASKLIST = ["tomato salad", "lettuce salad", "onion salad", "lettuce-tomato salad", "onion-tomato salad", "lettuce-onion salad", "lettuce-onion-tomato salad"]

class Overcooked_V1(gym.Env):

    """
    Overcooked Domain Description
    ------------------------------
    Agent with primitive actions ["right", "down", "left", "up"]
    TASKLIST = ["tomato salad", "lettuce salad", "onion salad", "lettuce-tomato salad", "onion-tomato salad", "lettuce-onion salad", "lettuce-onion-tomato salad"]

    1) Agent is allowed to pick up/put down food/plate on the counter;
    2) Agent is allowed to chop food into pieces if the food is on the cutting board counter;
    3) Agent is allowed to deliver food to the delivery counter;
    4) Only unchopped food is allowed to be chopped;
    """

    metadata = {
        'render.modes': ['human', 'rgb_array'],
        'video.frames_per_second' : 5
        }

    def __init__(self, grid_dim, task, rewardList, map_type = "A", n_agent = 2, obs_radius = 2, mode = "vector", debug = True):

        """
        Parameters
        ----------
        gird_dim : tuple(int, int)
            The size of the grid world([7, 7]/[9, 9]).
        task : int
            The index of the target recipe.
        rewardList : dictionary
            The list of the reward.
            e.g rewardList = {"subtask finished": 10, "correct delivery": 200, "wrong delivery": -5, "step penalty": -0.1}
        map_type : str
            The type of the map(A/B/C).
        n_agent: int
            The number of the agents.
        obs_radius: int
            The radius of the agents.
        mode: string
            The type of the observation(vector/image).
        debug : bool
            Whehter print the debug information.
        """
        self.action_space = spaces.Discrete(5)
        self.observation_space = spaces.Box(-5, 5, dtype=np.float32)


        self.xlen, self.ylen = grid_dim
        # if debug:
        self.game = Game(self)

        self.task = task
        self.rewardList = rewardList
        self.mapType = map_type
        self.debug = True
        self.n_agent = n_agent
        self.mode = mode
        self.obs_radius = obs_radius

        map = []

        if self.xlen == 7 and self.ylen == 7:
            if self.n_agent == 2:
                if self.mapType == "A":
                    map =  [[1, 1, 1, 1, 1, 3, 1],
                            [6, 0, 2, 0, 2, 0, 4],
                            [6, 0, 0, 0, 0, 0, 8],
                            [7, 0, 0, 0, 0, 0, 1],
                            [1, 0, 0, 0, 0, 0, 1],
                            [1, 0, 0, 0, 0, 0, 5],
                            [1, 1, 1, 1, 1, 5, 1]]
                elif self.mapType == "B":
                    map =  [[1, 1, 1, 1, 1, 3, 1],
                            [6, 0, 2, 1, 2, 0, 4],
                            [6, 0, 0, 1, 0, 0, 8],
                            [7, 0, 0, 1, 0, 0, 1],
                            [1, 0, 0, 1, 0, 0, 1],
                            [1, 0, 0, 1, 0, 0, 5],
                            [1, 1, 1, 1, 1, 5, 1]]
                elif self.mapType == "C":
                    map =  [[1, 1, 1, 1, 1, 3, 1],
                            [6, 0, 2, 1, 2, 0, 4],
                            [6, 0, 0, 1, 0, 0, 8],
                            [7, 0, 0, 1, 0, 0, 1],
                            [1, 0, 0, 1, 0, 0, 1],
                            [1, 0, 0, 0, 0, 0, 5],
                            [1, 1, 1, 1, 1, 5, 1]]
            elif self.n_agent == 3:
                if self.mapType == "A":
                    map =  [[1, 1, 1, 1, 1, 3, 1],
                            [6, 0, 2, 0, 2, 0, 4],
                            [6, 0, 0, 0, 0, 0, 8],
                            [7, 0, 0, 0, 0, 0, 1],
                            [1, 0, 0, 0, 0, 0, 1],
                            [1, 0, 2, 0, 0, 0, 5],
                            [1, 1, 1, 1, 1, 5, 1]]
                elif self.mapType == "B":
                    map =  [[1, 1, 1, 1, 1, 3, 1],
                            [6, 0, 2, 1, 2, 0, 4],
                            [6, 0, 0, 1, 0, 0, 8],
                            [7, 0, 0, 1, 0, 0, 1],
                            [1, 0, 0, 1, 0, 0, 1],
                            [1, 0, 2, 1, 0, 0, 5],
                            [1, 1, 1, 1, 1, 5, 1]]
                elif self.mapType == "C":
                    map =  [[1, 1, 1, 1, 1, 3, 1],
                            [6, 0, 2, 1, 2, 0, 4],
                            [6, 0, 0, 1, 0, 0, 8],
                            [7, 0, 0, 1, 0, 0, 1],
                            [1, 0, 0, 1, 0, 0, 1],
                            [1, 0, 2, 0, 0, 0, 5],
                            [1, 1, 1, 1, 1, 5, 1]]
        elif self.xlen == 9 and self.ylen == 9:
            if self.n_agent == 2:
                if self.mapType == "A":
                    map =  [[1, 1, 1, 1, 1, 1, 1, 3, 1],
                            [6, 0, 2, 0, 0, 0, 2, 0, 4],
                            [6, 0, 0, 0, 0, 0, 0, 0, 8],
                            [7, 0, 0, 0, 0, 0, 0, 0, 1],
                            [1, 0, 0, 0, 0, 0, 0, 0, 1],
                            [1, 0, 0, 0, 0, 0, 0, 0, 1],
                            [1, 0, 0, 0, 0, 0, 0, 0, 1],
                            [1, 0, 0, 0, 0, 0, 0, 0, 5],
                            [1, 1, 1, 1, 1, 1, 1, 5, 1]]
                elif self.mapType == "B":
                    map =  [[1, 1, 1, 1, 1, 1, 1, 3, 1],
                            [6, 0, 2, 0, 1, 0, 2, 0, 4],
                            [6, 0, 0, 0, 1, 0, 0, 0, 8],
                            [7, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 0, 0, 1, 0, 0, 0, 5],
                            [1, 1, 1, 1, 1, 1, 1, 5, 1]]
                elif self.mapType == "C":
                    map =  [[1, 1, 1, 1, 1, 1, 1, 3, 1],
                            [6, 0, 2, 0, 1, 0, 2, 0, 4],
                            [6, 0, 0, 0, 1, 0, 0, 0, 8],
                            [7, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 0, 0, 0, 0, 0, 0, 5],
                            [1, 1, 1, 1, 1, 1, 1, 5, 1]]
            elif self.n_agent == 3:
                if self.mapType == "A":
                    map =  [[1, 1, 1, 1, 1, 1, 1, 3, 1],
                            [6, 0, 2, 0, 0, 0, 2, 0, 4],
                            [6, 0, 0, 0, 0, 0, 0, 0, 8],
                            [7, 0, 0, 0, 0, 0, 0, 0, 1],
                            [1, 0, 0, 0, 0, 0, 0, 0, 1],
                            [1, 0, 0, 0, 0, 0, 0, 0, 1],
                            [1, 0, 0, 0, 0, 0, 0, 0, 1],
                            [1, 0, 2, 0, 0, 0, 0, 0, 5],
                            [1, 1, 1, 1, 1, 1, 1, 5, 1]]
                elif self.mapType == "B":
                    map =  [[1, 1, 1, 1, 1, 1, 1, 3, 1],
                            [6, 0, 2, 0, 1, 0, 2, 0, 4],
                            [6, 0, 0, 0, 1, 0, 0, 0, 8],
                            [7, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 2, 0, 1, 0, 0, 0, 5],
                            [1, 1, 1, 1, 1, 1, 1, 5, 1]]
                elif self.mapType == "C":
                    map =  [[1, 1, 1, 1, 1, 1, 1, 3, 1],
                            [6, 0, 2, 0, 1, 0, 2, 0, 4],
                            [6, 0, 0, 0, 1, 0, 0, 0, 8],
                            [7, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 0, 0, 1, 0, 0, 0, 1],
                            [1, 0, 2, 0, 0, 0, 0, 0, 5],
                            [1, 1, 1, 1, 1, 1, 1, 5, 1]]
        self.initMap = map
        self.map = copy.deepcopy(self.initMap)

        self.oneHotTask = []
        for t in TASKLIST:
            if t == self.task:
                self.oneHotTask.append(1)
            else:
                self.oneHotTask.append(0)

        self._createItems()
        self.n_agent = len(self.agent)

        #action: move(up, down, left, right), stay
        self.action_space = spaces.Discrete(5)

        #Observation: agent(pos[x,y]) dim = 2
        #    knife(pos[x,y]) dim = 2
        #    delivery (pos[x,y]) dim = 2
        #    plate(pos[x,y]) dim = 2
        #    food(pos[x,y]/status) dim = 3

        self._initObs()
        self.observation_space = spaces.Box(low=0, high=1, shape=(len(self._get_obs()[0]),), dtype=np.float32)
        self.obs_space = spaces.Box(low=0, high=1, shape=(len(np.concatenate(self._get_obs(),axis=None)),), dtype=np.float32)


    def _createItems(self):
        self.agent = []
        self.knife = []
        self.delivery = []
        self.tomato = []
        self.lettuce = []
        self.onion = []
        self.plate = []
        self.itemList = []
        agent_idx = 0
        for x in range(self.xlen):
            for y in range(self.ylen):
                if self.map[x][y] == ITEMIDX["agent"]:
                    self.agent.append(Agent(x, y, color = AGENTCOLOR[agent_idx]))
                    agent_idx += 1
                elif self.map[x][y] == ITEMIDX["knife"]:
                    self.knife.append(Knife(x, y))
                elif self.map[x][y] == ITEMIDX["delivery"]:
                    self.delivery.append(Delivery(x, y))
                elif self.map[x][y] == ITEMIDX["tomato"]:
                    self.tomato.append(Tomato(x, y))
                elif self.map[x][y] == ITEMIDX["lettuce"]:
                    self.lettuce.append(Lettuce(x, y))
                elif self.map[x][y] == ITEMIDX["onion"]:
                    self.onion.append(Onion(x, y))
                elif self.map[x][y] == ITEMIDX["plate"]:
                    self.plate.append(Plate(x, y))

        self.itemDic = {"tomato": self.tomato, "lettuce": self.lettuce, "onion": self.onion, "plate": self.plate, "knife": self.knife, "delivery": self.delivery, "agent": self.agent}
        for key in self.itemDic:
            self.itemList += self.itemDic[key]


    def _initObs(self):
        obs = []
        for item in self.itemList:
            obs.append(item.x / self.xlen)
            obs.append(item.y / self.ylen)
            if isinstance(item, Food):
                obs.append(item.cur_chopped_times / item.required_chopped_times)

        obs += self.oneHotTask

        for agent in self.agent:
            agent.obs = obs
        return [np.array(obs)] * self.n_agent


    def _get_vector_state(self):
        state = []
        for item in self.itemList:
            x = item.x / self.xlen
            y = item.y / self.ylen
            state.append(x)
            state.append(y)
            if isinstance(item, Food):
                state.append(item.cur_chopped_times / item.required_chopped_times)

        state += self.oneHotTask
        return [np.array(state)] * self.n_agent

    def _get_image_state(self):
        return [self.game.get_image_obs()] * self.n_agent

    def _get_obs(self):
        """
        Returns
        -------
        obs : list
            observation for each agent.
        """

        vec_obs = self._get_vector_obs()
        if self.obs_radius > 0:
            if self.mode == "vector":
                return vec_obs
            elif self.mode == "image":
                return self._get_image_obs()
        else:
            if self.mode == "vector":
                return self._get_vector_state()
            elif self.mode == "image":
                return self._get_image_state()

    def _get_vector_obs(self):

        """
        Returns
        -------
        vector_obs : list
            vector observation for each agent.
        """

        po_obs = []

        for agent in self.agent:
            obs = []
            idx = 0
            if self.xlen == 7 and self.ylen == 7:
                if self.mapType == "A":
                    agent.pomap= [[1, 1, 1, 1, 1, 1, 1],
                                  [1, 0, 0, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 0, 0, 1],
                                  [1, 1, 1, 1, 1, 1, 1]]
                elif self.mapType == "B":
                    agent.pomap= [[1, 1, 1, 1, 1, 1, 1],
                                  [1, 0, 0, 1, 0, 0, 1],
                                  [1, 0, 0, 1, 0, 0, 1],
                                  [1, 0, 0, 1, 0, 0, 1],
                                  [1, 0, 0, 1, 0, 0, 1],
                                  [1, 0, 0, 1, 0, 0, 1],
                                  [1, 1, 1, 1, 1, 1, 1]]
                elif self.mapType == "C":
                    agent.pomap= [[1, 1, 1, 1, 1, 1, 1],
                                  [1, 0, 0, 1, 0, 0, 1],
                                  [1, 0, 0, 1, 0, 0, 1],
                                  [1, 0, 0, 1, 0, 0, 1],
                                  [1, 0, 0, 1, 0, 0, 1],
                                  [1, 0, 0, 0, 0, 0, 1],
                                  [1, 1, 1, 1, 1, 1, 1]]
            elif self.xlen == 9 and self.ylen == 9:
                if self.mapType == "A":
                    agent.pomap= [[1, 1, 1, 1, 1, 1, 1, 1, 1],
                                  [1, 0, 0, 0, 0, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 0, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 0, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 0, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 0, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 0, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 0, 0, 0, 0, 1],
                                  [1, 1, 1, 1, 1, 1, 1, 1, 1]]
                elif self.mapType == "B":
                    agent.pomap= [[1, 1, 1, 1, 1, 1, 1, 1, 1],
                                  [1, 0, 0, 0, 1, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 1, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 1, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 1, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 1, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 1, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 1, 0, 0, 0, 1],
                                  [1, 1, 1, 1, 1, 1, 1, 1, 1]]
                elif self.mapType == "C":
                    agent.pomap= [[1, 1, 1, 1, 1, 1, 1, 1, 1],
                                  [1, 0, 0, 0, 1, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 1, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 1, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 1, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 1, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 1, 0, 0, 0, 1],
                                  [1, 0, 0, 0, 0, 0, 0, 0, 1],
                                  [1, 1, 1, 1, 1, 1, 1, 1, 1]]

            for item in self.itemList:
                if item.x >= agent.x - self.obs_radius and item.x <= agent.x + self.obs_radius and item.y >= agent.y - self.obs_radius and item.y <= agent.y + self.obs_radius \
                    or self.obs_radius == 0:
                    x = item.x / self.xlen
                    y = item.y / self.ylen
                    obs.append(x)
                    obs.append(y)
                    idx += 2
                    if isinstance(item, Food):
                        obs.append(item.cur_chopped_times / item.required_chopped_times)
                        idx += 1
                else:
                    x = agent.obs[idx] * self.xlen
                    y = agent.obs[idx + 1] * self.ylen
                    if x >= agent.x - self.obs_radius and x <= agent.x + self.obs_radius and y >= agent.y - self.obs_radius and y <= agent.y + self.obs_radius:
                        x = item.initial_x
                        y = item.initial_y
                    x = x / self.xlen
                    y = y / self.ylen

                    obs.append(x)
                    obs.append(y)
                    idx += 2
                    if isinstance(item, Food):
                        obs.append(agent.obs[idx] / item.required_chopped_times)
                        idx += 1

                agent.pomap[int(x * self.xlen)][int(y * self.ylen)] = ITEMIDX[item.rawName]
            agent.pomap[agent.x][agent.y] = ITEMIDX["agent"]

            obs += self.oneHotTask
            agent.obs = obs
            po_obs.append(np.array(obs))
        return po_obs

    def _get_image_obs(self):

        """
        Returns
        -------
        image_obs : list
            image observation for each agent.
        """

        po_obs = []
        frame = self.game.get_image_obs()
        old_image_width, old_image_height, channels = frame.shape
        new_image_width = int((old_image_width / self.xlen) * (self.xlen + 2 * (self.obs_radius - 1)))
        new_image_height =  int((old_image_height / self.ylen) * (self.ylen + 2 * (self.obs_radius - 1)))
        color = (0,0,0)
        obs = np.full((new_image_height,new_image_width, channels), color, dtype=np.uint8)

        x_center = (new_image_width - old_image_width) // 2
        y_center = (new_image_height - old_image_height) // 2

        obs[x_center:x_center+old_image_width, y_center:y_center+old_image_height] = frame

        for idx, agent in enumerate(self.agent):
            agent_obs = self._get_PO_obs(obs, agent.x, agent.y, old_image_width, old_image_height)
            po_obs.append(agent_obs)
        return po_obs

    def _get_PO_obs(self, obs, x, y, ori_width, ori_height):
        x1 = (x - 1) * int(ori_width / self.xlen)
        x2 = (x + self.obs_radius * 2) * int(ori_width / self.xlen)
        y1 = (y - 1) * int(ori_height / self.ylen)
        y2 = (y + self.obs_radius * 2) * int(ori_height / self.ylen)
        return obs[x1:x2, y1:y2]

    def _findItem(self, x, y, itemName):
        for item in self.itemDic[itemName]:
            if item.x == x and item.y == y:
                return item
        return None

    @property
    def state_size(self):
        return self.get_state().shape[0]

    @property
    def obs_size(self):
        return [self.observation_space.shape[0]] * self.n_agent

    @property
    def n_action(self):
        return [a.n for a in self.action_spaces]

    @property
    def action_spaces(self):
        return [self.action_space] * self.n_agent

    def get_avail_actions(self):
        return [self.get_avail_agent_actions(i) for i in range(self.n_agent)]

    def get_avail_agent_actions(self, nth):
        return [1] * self.action_spaces[nth].n

    def action_space_sample(self, i):
        return np.random.randint(self.action_spaces[i].n)

    def reset(self):

        """
        Returns
        -------
        obs : list
            observation for each agent.
        """

        self.map = copy.deepcopy(self.initMap)
        self._createItems()
        self._initObs()
        # if self.debug:
        self.game.on_cleanup()

        return self._get_obs()

    def step(self, action):


        """
        Parameters
        ----------
        action: list
            action for each agent

        Returns
        -------
        obs : list
            observation for each agent.
        rewards : list
        terminate : list
        info : dictionary
        """

        reward = self.rewardList["step penalty"]
        done = False
        info = {}
        info['cur_mac'] = action
        info['mac_done'] = [True] * self.n_agent
        info['collision'] = []

        all_action_done = False

        for agent in self.agent:
            agent.moved = False

        # if self.debug:
        # print("in overcooked primitive actions:", action)

        while not all_action_done:
            for idx, agent in enumerate(self.agent):

                agent_action = action[idx]
                if agent.moved:
                    continue
                agent.moved = True

                if agent_action < 4:

                    target_x = agent.x + DIRECTION[agent_action][0]
                    target_y = agent.y + DIRECTION[agent_action][1]
                    target_name = ITEMNAME[self.map[target_x][target_y]]

                    if target_name == "agent":
                        target_agent = self._findItem(target_x, target_y, target_name)
                        if not target_agent.moved:
                            agent.moved = False
                            target_agent_action = action[AGENTCOLOR.index(target_agent.color)]
                            if target_agent_action < 4:
                                new_target_agent_x = target_agent.x + DIRECTION[target_agent_action][0]
                                new_target_agent_y = target_agent.y + DIRECTION[target_agent_action][1]
                                if new_target_agent_x == agent.x and new_target_agent_y == agent.y:
                                    target_agent.move(new_target_agent_x, new_target_agent_y)
                                    agent.move(target_x, target_y)
                                    agent.moved = True
                                    target_agent.moved = True
                    elif  target_name == "space":
                        self.map[agent.x][agent.y] = ITEMIDX["space"]
                        agent.move(target_x, target_y)
                        self.map[target_x][target_y] = ITEMIDX["agent"]
                    #pickup and chop
                    elif not agent.holding:
                        if target_name == "tomato" or target_name == "lettuce" or target_name == "plate" or target_name == "onion":
                            item = self._findItem(target_x, target_y, target_name)
                            agent.pickup(item)
                            self.map[target_x][target_y] = ITEMIDX["counter"]
                        elif target_name == "knife":
                            knife = self._findItem(target_x, target_y, target_name)
                            if isinstance(knife.holding, Plate):
                                item = knife.holding
                                knife.release()
                                agent.pickup(item)
                            elif isinstance(knife.holding, Food):
                                if knife.holding.chopped:
                                    item = knife.holding
                                    knife.release()
                                    agent.pickup(item)
                                else:
                                    knife.holding.chop()
                                    if knife.holding.chopped:

                                        if knife.holding.rawName in self.task:
                                            reward += self.rewardList["subtask finished"]
                    #put down
                    elif agent.holding:
                        if target_name == "counter":
                            if agent.holding.rawName in ["tomato", "lettuce", "onion", "plate"]:
                                self.map[target_x][target_y] = ITEMIDX[agent.holding.rawName]
                            agent.putdown(target_x, target_y)
                        elif target_name == "plate":
                            if isinstance(agent.holding, Food):
                                if agent.holding.chopped:
                                    plate = self._findItem(target_x, target_y, target_name)
                                    item = agent.holding
                                    agent.putdown(target_x, target_y)
                                    plate.contain(item)

                        elif target_name == "knife":
                            knife = self._findItem(target_x, target_y, target_name)
                            if not knife.holding:
                                item = agent.holding
                                agent.putdown(target_x, target_y)
                                knife.hold(item)
                            elif isinstance(knife.holding, Food) and isinstance(agent.holding, Plate):
                                item = knife.holding
                                if item.chopped:
                                    knife.release()
                                    agent.holding.contain(item)
                            elif isinstance(knife.holding, Plate) and isinstance(agent.holding, Food):
                                plate_item = knife.holding
                                food_item = agent.holding
                                if food_item.chopped:
                                    knife.release()
                                    agent.pickup(plate_item)
                                    agent.holding.contain(food_item)
                        elif target_name == "delivery":
                            if isinstance(agent.holding, Plate):
                                if agent.holding.containing:
                                    dishName = ""
                                    foodList = [Lettuce, Onion, Tomato]
                                    foodInPlate = [-1] * len(foodList)

                                    for f in range(len(agent.holding.containing)):
                                        for i in range(len(foodList)):
                                            if isinstance(agent.holding.containing[f], foodList[i]):
                                                foodInPlate[i] = f
                                    for i in range(len(foodList)):
                                        if foodInPlate[i] > -1:
                                            dishName += agent.holding.containing[foodInPlate[i]].rawName + "-"
                                    dishName = dishName[:-1] + " salad"
                                    if dishName == self.task:
                                        item = agent.holding
                                        agent.putdown(target_x, target_y)
                                        self.delivery[0].hold(item)
                                        reward += self.rewardList["correct delivery"]
                                        done = True
                                    else:
                                        reward += self.rewardList["wrong delivery"]
                                        item = agent.holding
                                        agent.putdown(target_x, target_y)
                                        food = item.containing
                                        item.release()
                                        item.refresh()
                                        self.map[item.x][item.y] = ITEMIDX[item.name]
                                        for f in food:
                                            f.refresh()
                                            self.map[f.x][f.y] = ITEMIDX[f.rawName]
                                else:
                                    reward += self.rewardList["wrong delivery"]
                                    plate = agent.holding
                                    agent.putdown(target_x, target_y)
                                    plate.refresh()
                                    self.map[plate.x][plate.y] = ITEMIDX[plate.name]
                            else:
                                reward += self.rewardList["wrong delivery"]
                                food = agent.holding
                                agent.putdown(target_x, target_y)
                                food.refresh()
                                self.map[food.x][food.y] = ITEMIDX[food.rawName]

                        elif target_name in ["tomato", "lettuce", "onion"]:
                            item = self._findItem(target_x, target_y, target_name)
                            if item.chopped and isinstance(agent.holding, Plate):
                                agent.holding.contain(item)
                                self.map[target_x][target_y] = ITEMIDX["counter"]

            all_action_done = True
            for agent in self.agent:
                if agent.moved == False:
                    all_action_done = False

        return self._get_obs(), [reward] * self.n_agent, done, info

    def render(self, mode='human'):
        return self.game.on_render()




In [36]:
# @title Initialization for environment

env_id='Overcooked-v1'
n_agent=2
grid_dim=[7, 7]
task_id=6
map_type='A'
obs_radius=2
mode='vector'

rewardList = {"subtask finished": 10, "correct delivery": 200, "wrong delivery": -5, "step penalty": -0.1}
TASKLIST = ["tomato salad", "lettuce salad", "onion salad", "lettuce-tomato salad", "onion-tomato salad", "lettuce-onion salad", "lettuce-onion-tomato salad"]

env=Overcooked_V1(grid_dim, TASKLIST[task_id], rewardList, map_type = "B", n_agent = 2, obs_radius = 2, mode = "vector", debug = False)


In [37]:
# @title Printing the outputs
def print_out(obs,reward,actionName,step,n_agent,info):
  print("------------------------------------------------------------------")
  print("step: ", step, "rewards: ", reward)
  print("#############################################")

  Actiontxts=["Blue Agent","Pink Agent","Green Agent"]
  for i in range(n_agent):
        print(Actiontxts[i]," Action: ", actionName[info['cur_mac'][i]])
        print("Action Done: ", info['mac_done'][i])
        print(Actiontxts[i]," Observation")
        print("tomato pos: ", obs[i][0:2]*7)
        print("tomato status: ", obs[i][2])
        print("lettuce pos: ", obs[i][3:5]*7)
        print("lettuce status: ", obs[i][5])
        print("onion pos: ", obs[i][6:8]*7)
        print("onion status: ", obs[i][8])
        print("plate-1 pos: ", obs[i][9:11]*7)
        print("plate-2 pos: ", obs[i][11:13]*7)
        print("knife-1 pos: ", obs[i][13:15]*7)
        print("knife-2 pos: ", obs[i][15:17]*7)
        print("delivery: ", obs[i][17:19]*7)
        print("agent-1: ", obs[i][19:21]*7)
        print("agent-2: ", obs[i][21:23]*7)
        print("agent-3: ", obs[i][23:25]*7)
        print("order: ", obs[i][25:])
        print("#############################################")
        print("#############################################")
        print()
        print()
        print()

In [38]:
# @title Play
import pandas as pd
import argparse
import gym
from IPython.display import clear_output
# from gym_macro_overcooked.macActEnvWrapper import MacEnvWrapper

TASKLIST = ["tomato salad", "lettuce salad", "onion salad", "lettuce-tomato salad", "onion-tomato salad", "lettuce-onion salad", "lettuce-onion-tomato salad"]

def play(env, grid_dim, task, map_type, obs_radius, mode, debug,show_output):
    n_agent=env.n_agent

    rewardList = {"subtask finished": 10, "correct delivery": 200, "wrong delivery": -5, "step penalty": -0.1}
    env_params = {'grid_dim': grid_dim,
                    'task': TASKLIST[task],
                    'rewardList': rewardList,
                    'map_type': map_type,
                    'n_agent': n_agent,
                    'obs_radius': obs_radius,
                    'mode': mode,
                    'debug': debug
                }

    #               0        1       2        3     4
    actionName = ["stay", "right", "down", "left", "up"]

    rewards = 0
    discount = 1
    step = 0

    obs = env.reset()
    # env.render()

    action1_data=[]
    action2_data=[]
    target=[]
    step=0
    while step<200:


        # a = input("input:").split(" ")
        # action = [int(a[0]), int(a[1]), int(a[2])]
        action=np.random.randint(5,size=2)
        obs, reward, done, info = env.step(action)

        action_explanation="Agent"+str(1)+' selects '+actionName[action[0]]
        for i in range(1,n_agent):
          action_explanation=action_explanation+" and Agent"+str(i+1)+' selects '+actionName[action[i]]
        action1_data.append(action[0])
        action2_data.append(action[1])
        target.append(action_explanation)

        # print("in overcooked primitive actions:", action_explanation)
        # print("info",info)
        # print(env.task)


        if show_output==1:
          clear_output(wait=True)
          img_rgb=env.render()
          plt.imshow(img_rgb)
          plt.pause(0.0001)  # Small pause to update the plot





        rewards += discount * reward[0]
        discount *= 0.99

        if show_output==1:
          print_out(obs,reward,actionName,step,n_agent,info)
        step += 1
        if done:
            break
    return action1_data,action2_data,target




grid_dim=[7, 7]
task=6
map_type='A'
obs_radius=2
mode='vector'
debug=False
show_output=0
action1_data,action2_data,target=play(env, grid_dim, task, map_type, obs_radius, mode, debug,show_output)
df = pd.DataFrame({'action1_data':action1_data,
                   'action2_data':action2_data,
                   'target':target
                   })
df.to_csv('experience_data.csv', index=False)



In [39]:
# @title Training
def _plot(
    agent,
    frame_idx: int,
    scores: List[float],
    losses: List[float],
    epsilons: List[float],

):

    """Plot the training progresses."""
    clear_output(True)
    plt.close()
    plt.figure(figsize=(16, 4))
    # plt.figure()
    plt.subplot(131)
    # plt.title('frame %s. score: %s' % (frame_idx, np.mean(scores[-10:])))
    plt.plot(scores)
    plt.xlabel('iteration')
    plt.ylabel('Cumulative Reward')
    plt.subplot(132)
    # plt.figure()
    # plt.title('loss')
    plt.plot(losses)
    plt.xlabel('iteration')
    plt.ylabel('Loss')
    plt.subplot(133)
    # plt.figure()
    # plt.title('epsilon')
    plt.plot(epsilons)
    plt.xlabel('iteration')
    plt.ylabel('epsilon')
    plt.show()
def print_output(agent,print_mode):
      if print_mode==1:
        for i in range(0,1):
          print("-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.-.")


def training(agent1,agent2,agent3, eps_method,max_steps,num_frames, plotting_interval,print_mode):
    """Train the agent."""


    state  = agent1.env.reset()
    update_cnt = 0
    epsilons = []
    losses = []
    scores = []
    score = 0
    done=False
    rewards=[]
    actions=[]
    states=[]
    for frame_idx in range(0, num_frames + 1):

        state  = agent1.reset()



        score=[]
        stp=0
        los=[]

        while done==False  and stp<max_steps:

          action1 = agent1.select_action(state)
          action2 = agent2.select_action(state)
          action3 = agent3.select_action(state)
          action=[action1,action3]
          print(action)


          next_state, reward, done,_= agent1.step(action)

          rewards.append(reward)
          states.append(states)
          actions.append(actions)

          state = next_state
          score.append(reward)


          actor_loss, critic_loss = agent3.update_model(next_state)
          # actor_losses.append(actor_loss)
          # critic_losses.append(critic_loss)
          stp+=1


        # if training is ready
        if len(agent1.memory) >= agent1.batch_size:

            loss = agent1.update_model()
            los.append(loss)
            update_cnt += 1

            # linearly decrease epsilon
            #Eps 1    : max(min_e,e-(max_e-min_e)*eps_d)
            if eps_method==1:

              agent1.epsilon = max(
                  agent1.min_epsilon, agent1.epsilon - (
                      agent1.max_epsilon - agent1.min_epsilon
                  ) * agent1.epsilon_decay
              )
            else:
              #Eps2  : max(min_e,e*eps_d)
              agent1.epsilon = max(agent1.min_epsilon, agent1.epsilon* agent1.epsilon_decay)

            epsilons.append(agent1.epsilon)


            # if hard update is needed
            if update_cnt % agent1.target_update == 0:
                agent1._target_hard_update()
        # if frame_idx>0 and frame_idx%35==0:
        # agent2.update_policy( rewards, states, actions)
        # plotting
        if frame_idx % plotting_interval == 0:
            _plot(agent1,frame_idx, scores, losses, epsilons)
            print_output(agent1,print_mode=1)
        scores.append(np.mean(score))
        losses.append(np.mean(los))


def test(agent):
    """Train the agent."""
    agent.is_test = True


    update_cnt = 0
    epsilons = []
    losses = []
    scores = []
    score = 0
    done=0
    state= agent.env.reset()
    score=0
    for step in range(0,max_steps):
      agent.env.stp=step
      action = agent.select_action(state)
      next_state, reward, done,_= agent.step(action)
      state = next_state
      score += reward
    print("score: ", score)
    print_output(print_mode=1)





In [40]:
# @title Runing Training
Train=1
if Train==1:
  max_steps=200
  max_episode=10000

  plotting_interval=10
  print_mode=0
  eps_method=2
  state_size = env.obs_space.shape[0]
  action_size = env.action_space.n
  # parameters

  memory_size = 100000 #*****************************************
  batch_size = 128  # 128
  target_update = 100
  epsilon_decay =0.999 #**********************************************************************

  dqn_agent = DQNAgent(env, memory_size, batch_size, target_update, epsilon_decay)
  pg_agent = PolicyGradientAgent(env)
  ppo_agent=PPOAgent(env,gamma = 0.9,tau = 0.8,batch_size = 64,epsilon = 0.2,epoch = 64,rollout_len = 2048,entropy_weight = 0.005)
  training(dqn_agent,pg_agent,ppo_agent, eps_method,max_steps,max_episode, plotting_interval,print_mode)

cpu
out_dim 5
out_dim 5
cpu
[2, array([-0.00118555, -0.00416617, -0.00113694,  0.0005507 ,  0.00178603],
      dtype=float32)]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
from collections import deque
import random

# Set up the environment

state_size = env.obs_space.shape[0]
action_size = env.action_space.n

# Initialize the agents
ppo_agent = PPOAgent(env, state_size, action_size)
pg_agent = PolicyGradientAgent(env, state_size, action_size)
dqn_agent = DQNAgent(env, state_size, action_size)

# Training hyperparameters
num_episodes = 1000
max_steps = 200
batch_size = 32
ppo_update_iters = 10
ppo_clip_range = 0.2

# Training loop
for episode in range(num_episodes):
    state = env.reset()
    total_reward = 0

    ppo_states, ppo_actions, ppo_rewards, ppo_dones = [], [], [], []
    pg_states, pg_actions, pg_rewards = [], [], []
    dqn_states, dqn_actions, dqn_rewards, dqn_next_states, dqn_dones = [], [], [], [], []

    for step in range(max_steps):
        # Select actions and take a step
        ppo_action = ppo_agent.select_action(state)
        pg_action = pg_agent.select_action(state)
        dqn_action = dqn_agent.select_action(state)
        next_state, reward, done, _ = env.step(ppo_action)
        total_reward += reward

        # Store the transitions
        ppo_states.append(state)
        ppo_actions.append(ppo_action)
        ppo_rewards.append(reward)
        ppo_dones.append(done)

        pg_states.append(state)
        pg_actions.append(pg_action)
        pg_rewards.append(reward)

        dqn_states.append(state)
        dqn_actions.append(dqn_action)
        dqn_rewards.append(reward)
        dqn_next_states.append(next_state)
        dqn_dones.append(done)

        if done:
            break

        state = next_state

    print(f'Episode {episode+1}/{num_episodes}, Total Reward: {total_reward}')

    # Update the agents
    ppo_agent.update(ppo_states, ppo_actions, ppo_rewards, ppo_dones, ppo_update_iters, ppo_clip_range)
    pg_agent.update(pg_states, pg_actions, pg_rewards)

    if len(dqn_states) >= batch_size:
        dqn_agent.update(batch_size)

    # Evaluate the agents
    ppo_agent_score = evaluate_agent(ppo_agent, env, 10)
    pg_agent_score = evaluate_agent(pg_agent, env, 10)
    dqn_agent_score = evaluate_agent(dqn_agent, env, 10)
    print(f'PPO Agent Score: {ppo_agent_score}, PG Agent Score: {pg_agent_score}, DQN Agent Score: {dqn_agent_score}')

def evaluate_agent(agent, env, num_episodes):
    total_reward = 0
    for _ in range(num_episodes):
        state = env.reset()
        done = False
        while not done:
            if isinstance(agent, PPOAgent):
                action = agent.select_action(state, deterministic=True)
            else:
                action = agent.select_action(state)
            state, reward, done, _ = env.step(action)
            total_reward += reward
    return total_reward / num_episodes

class PPOAgent:
    # PPO agent implementation

class PolicyGradientAgent:
    # Policy Gradient agent implementation

class DQNAgent:
    # DQN agent implementation

## Test

In [ ]:
# Source_of_File="PC"
Source_of_File="Colab"
if Source_of_File=="PC" :
  uploaded=files.upload()
  filename = next(iter(uploaded))

In [ ]:

Test=1

if Test==1 :
  if Source_of_File=="PC" :
    with open('/content/'+filename , "rb") as f:
      Agent = pickle.load(f)

  else:
    Agent=agent
  Agent.max_it=max_steps
  Agent.test()